In [1]:
import pandas as pd
import numpy as np
import numpy.random as random
from route_functions import *

### Variables
We have our system state defined as  - 
* SS: (n, nd, (ch1, st1, r1,m, tbm, sbm), (ch2, st2, r2,m, tb2,m, sb2,m),.. (chn, stn, rn,m, tbn,m, sbn,m))
    * n - total number of available buses
    * m - subscript that denotes route m
    * nd - number of buses deployed
    * chn -  charging state of bus n
    * stn - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * rn,m - the route on which the bus is deployed
    * tbn,m - the previous stop the bus travelled to (travel time)
    * sbn,m - the previous stop which was served by the bus (service time)
        * If tbn= sbn, generate travel time for tbn+1; tbn = tbn+1 if accepted
        * Else if tbn> sbn, generate service time for sbn+1; sbn = sbn+1 if accepted

* t - the clock time/current time
* nrm - number of buses required on route m
* btn - time array for monitoring travel time and service time related to each bus


### Pseudo-code
1. Start at t = 0
2. Generate requirements from route, nd (required number of buses):
    1. Check availability of buses & minimum fuel level/battery capacity required for the routes
    2. Choose number of buses, K, to be deployed based on checks
        1. If K<nd, deploy buses with higher fuel/battery level and recharge/refuel the rest
        2. Else, send buses for recharge/refuel
    3. For each bus k, in {1, 2,.. n}, after selecting a specific route m {stops - 1, 2,... M}:
    4. Initialize j = 0 (Depot_Start)
    5. Generate interarrival times for stop j, update time tbk,j
    6. Generate service time at stop j, update time sbk,j
    7. STOP if j = M; Else update j = j + 1, goto 2.b. 
3. Based on the time we update the System State (SS) and the concerned variables.

### Cases
1. We generate demand for the route at time t,
    Sub-task: Generating series of buses to be deployed
    1. Condition 1: number of buses required <= number of charged/fuelled buses
        1. Deploy buses based on descending order of fuel level/charge
        2. Send buses for refuelling/recharging, if they are below a certain threshold
    2. Condition 2: number of buses required > number of charged/fuelled buses
        1. Send buses for refuelling/recharging
        2. As soon as the bus finishes recharging/refueling, buses get sent to the route
2. Routing of buses:
    1. Route 0: based on type of the bus and refuelling need. Will be a refuelling/ recharging route for the bus
    2. Route 1 to m: Buses will go through the stops till they reach Depot_Stop


## Variables
#### Buses
* bus: ($ch_n, st_n, r_{n,m}, tb_{n,m}, sb_{n,m}$)
    * $ch_n$ - charging state of bus n
    * $st_n$ - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * $r_{n,m}$ - the route on which the bus is deployed
    * $tb_{n,m}$ - the previous stop the bus travelled to (travel time)
    * $sb_{n,m}$ - the previous stop which was served by the bus (service time)
        * If $tb_n$= $sb_n$, generate travel time for $tb_{n+1}$; $tb_n$ = $tb_{n+1}$ if accepted
        * Else if $tb_n$> $sb_n$, generate service time for $sb_{n+1}$; $sb_n$ = $sb_{n+1}$ if accepted
    * Arrival time - route generated arrays
    * Service Time - route generated arrays
    * 

In [2]:
route_t = pd.read_excel('Model Parameters.xlsx', 'Routes')
route_t

,recharge_index,recharge_mean,recharge_std,refill_index,refill_mean,refill_std,route_1_index,route_1_mean,route_1_std,route_2_index,route_2_mean,route_2_std
0,0.0,2.0,0.25,0.0,1.5,0.25,0,5.00,1.00,0.0,5.00,1.0
1,1.0,12.0,5.00,1.0,5.0,2.00,1,1.00,0.50,1.0,1.00,0.5
2,0.0,1.0,0.50,0.0,2.0,0.50,0,7.00,2.00,0.0,7.00,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,1,0.75,1.50,1.0,0.75,1.5
4,NaN,NaN,NaN,NaN,NaN,NaN,0,5.00,1.50,0.0,5.00,1.5
5,NaN,NaN,NaN,NaN,NaN,NaN,1,1.20,0.75,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,0,4.00,2.00,NaN,NaN,NaN


In [3]:
print(route_time(route_t, 'recharge', 0))
print(route_time(route_t, 'refill', 0))
print(route_time(route_t, '2', 0))

([1.4796, 83.7489, 84.0103], array([0., 1., 0.]))
([0.3129, 18.7119, 19.5128], array([0., 1., 0.]))
([6.5906, 6.9761, 17.909, 19.3931, 24.1561], array([0., 1., 0., 1., 0.]))


C:\Users\krishrao\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [4]:
n_buses = 1
buses = [bus() for i in range(n_buses)]
t = 10
buses[0].assign_route(route_t, 'refill', t)
print(buses[0].time_arr)
print(buses[0].event_arr)

[10.2473, 16.2654, 16.8466]
[0. 1. 0.]


In [5]:
print(buses[0].next_t())
print(buses[0].next_e())

10.2473
0.0
